In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import torch
import itertools

In [3]:
from dblm.rva.discrete.tree_belief_propagation import TreeBeliefPropagation
adjacency_matrix = torch.tensor([
    [1,0,0,0],
    [1,1,0,0],
    [1,0,1,0],
    [0,0,1,1]
])
f1 = torch.log(torch.tensor(
    [0.1,0.3,0.2,0.1,0.05,0.0,0.0,0.25] + [0.1,0.3,0.2,0.1,0.05,0.0,0.0,0.25]
)).reshape(2,2,2,2)
f2 = torch.log(torch.tensor(
    [1, 1.] + [1, 3.]
)).reshape(2,2 )
f3 = torch.log(torch.tensor(
    [4, 1., 1., 4] + [4, 1., 1., 4]
)).reshape(2,2,2)
f4 = torch.log(torch.tensor(
    [2 * 1.3, 8 * 1.3] + [2 * 1.3, 8 * 1.3]
)).reshape(2, 2)
tbp = TreeBeliefPropagation()
log_phis = [f1, f2, f3, f4]
nvals = [2,2,2,2]
results = tbp.infer(adjacency_matrix, nvals, log_phis)
m = tbp.marginals(results)
logZ = tbp.log_partition(adjacency_matrix, log_phis, results)

In [4]:
ent = tbp.entropy(adjacency_matrix, log_phis, results)
marginals_factors = tbp.marginals_factors(adjacency_matrix, log_phis, results)

In [5]:
logZ = ent + sum((marginals_factors[i].exp() * (log_phis[i]).clamp(-1e10)).reshape(ent.size(0), -1).sum(-1) for i in range(len(marginals_factors)))

In [13]:
logps=[]
for a,b,c,d in itertools.product(*([0,1] for _ in range(4))):
     logps.append(tbp.energy(adjacency_matrix, nvals, [log_phi[0] for log_phi in log_phis], torch.tensor([[a,b,c,d]])))
logZ1 = torch.cat(logps).logsumexp(-1)
print(logZ,logZ1)

tensor([3.5838, 4.3211]) tensor(3.5838)


In [7]:
list([log_phi[0] for log_phi in log_phis][0].size()[:-adjacency_matrix[:,0].sum().item()])

[]